In [0]:
%run ./00_config

### bronze

In [0]:
spark.sql(f"""
-- 決算書ボリュームのファイルの文字を抽出し、bronzeテーブルを作成
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.settlement_bronze AS
SELECT
  path,
  ai_parse_document(content) AS content
FROM
  READ_FILES(
    '/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/pdf',
    format => 'binaryFile'
  );
""")

### silver

In [0]:
prompt = (
    '貴方はテキスト整理のプロフェッショナルです。決算についてJSON形式でまとめてください。parse_jsonできるように余計な文字列は入れないでください。'
    'それぞれのセグメントごとに売上高、営業利益、対前年同期売上増減率、概況状況をまとめて一つのテキストに整形してください。'
    'Keyはそれぞれ\"セグメント\"、\"売上高\"、\"営業利益\"、\"対前年同期売上増減率\"、\"概況\"でお願いします。'
    '[で始まり、]で終わるJSONで返してください。```json```という文字列は抜いてください。'
)

spark.sql(f"""
-- bronzeテーブルから必要項目を切り出して、Silverをテーブルを作成
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.settlement_silver AS
SELECT
  path,
  ai_query(
    'databricks-claude-sonnet-4',
    '{prompt}' || content
  ) AS summary
FROM
  {MY_CATALOG}.{MY_SCHEMA}.settlement_bronze
""")

In [0]:
spark.sql(f"""
-- json形式に格納されたSilverテーブルの日付、温度の列を展開して、Goldテーブルを作成
CREATE OR REPLACE TABLE {MY_CATALOG}.{MY_SCHEMA}.settlement_gold AS
SELECT
  uuid() AS id,
  regexp_extract(
    split(path, '/')[size(split(path, '/'))-1],
    '^(.*)\.',
    1
  ) AS `company`,
  split(path, '/')[5] AS Industry,
  exploded_case.`セグメント` AS `segment`,
  try_cast(
    regexp_replace(
      regexp_extract(exploded_case.`売上高`, '([0-9,.]+)', 1),
      ',',
      ''
    ) AS DOUBLE
  ) * CASE WHEN exploded_case.`売上高` LIKE '%億%' THEN 100 ELSE 1 END AS `Sales`,
  try_cast(
    regexp_replace(
      regexp_extract(exploded_case.`営業利益`, '([0-9,.]+)', 1),
      ',',
      ''
    ) AS DOUBLE
  ) * CASE WHEN exploded_case.`営業利益` LIKE '%億%' THEN 100 ELSE 1 END AS `profit`,
  exploded_case.`対前年同期売上増減率` AS yoy_change,
  exploded_case.`概況` AS summary
FROM
  {MY_CATALOG}.{MY_SCHEMA}.settlement_silver
LATERAL VIEW
  EXPLODE(
    from_json(
      summary,
      'ARRAY<STRUCT<`セグメント`:STRING,`売上高`:STRING,`営業利益`:STRING,`対前年同期売上増減率`:STRING,`概況`:STRING>>'
    )
  ) exploded_cases AS exploded_case;
""")

In [0]:
spark.sql(f"""
-- json形式に格納されたSilverテーブルの日付、温度の列を展開して、Goldテーブルを作成
ALTER TABLE {MY_CATALOG}.{MY_SCHEMA}.settlement_gold
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);
""")